In [7]:
from qiskit_cartan.cartan_plugin import *

import logging
logging.basicConfig(level='DEBUG')

#### Check `synth_cartan` works as intended with an example Heisenberg Hamiltonian

In [8]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.opflow import I, X, Y, Z
test_heisenberg_ham = (X^X^I) + (Y^Y^I) + (Z^Z^I) + (I^X^X) + (I^Y^Y) + (I^Z^Z)
test_time_evolve = 3
test_paulievolutiongate = PauliEvolutionGate(test_heisenberg_ham, time=test_time_evolve)
qc = synth_cartan(test_paulievolutiongate)
print(qc)

Optimization terminated successfully.
         Current function value: -2.303234
         Iterations: 15
         Function evaluations: 18
         Gradient evaluations: 18
--- 0.0714879035949707 seconds ---
Optimization Error:
3.878611739499617e-12
     ┌───────────────────────┐┌─────────────────────────┐»
q_0: ┤0                      ├┤0                        ├»
     │                       ││                         │»
q_1: ┤1 exp(-it YZX)(1.1781) ├┤1 exp(-it ZYX)(-0.49347) ├»
     │                       ││                         │»
q_2: ┤2                      ├┤2                        ├»
     └───────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                        ├┤0                        ├»
«     │                         ││                         │»
«q_1: ┤1 exp(-it XZY)(-0.47282) ├┤1 exp(-it ZXY)(-0.30774) ├»
«     │                         ││                         │»
«q_2: ┤2                   

In [9]:
from qiskit import Aer, transpile

unitary_simulator = Aer.get_backend('unitary_simulator')
qc_transp = transpile(qc, unitary_simulator)
final_unitary_cartan = unitary_simulator.run(qc_transp).result().get_unitary()

import scipy
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], 
                  np.array([1, 1, 1, 1, 1, 1])).to_matrix()
propagator = scipy.linalg.expm(-1j*H*test_time_evolve)
matrix_norm = np.linalg.norm(propagator - final_unitary_cartan)
print("\n")
print("matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: \n", matrix_norm)

DEBUG:stevedore.extension:found extension EntryPoint(name='basic', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:BasicSwapPassManager', group='qiskit.transpiler.routing')
DEBUG:stevedore.extension:found extension EntryPoint(name='lookahead', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:LookaheadSwapPassManager', group='qiskit.transpiler.routing')
DEBUG:stevedore.extension:found extension EntryPoint(name='none', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:NoneRoutingPassManager', group='qiskit.transpiler.routing')
DEBUG:stevedore.extension:found extension EntryPoint(name='sabre', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:SabreSwapPassManager', group='qiskit.transpiler.routing')
DEBUG:stevedore.extension:found extension EntryPoint(name='stochastic', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:StochasticSwapPassManager', group='qiskit.transpiler.routing')
DEBUG:stevedore.extension:found extension EntryPoin



matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: 
 7.769560961856694e-07


#### Check our plugin is installed

In [10]:
from qiskit.transpiler.passes.synthesis.plugin import HighLevelSynthesisPluginManager
HLS_plugin_manager = HighLevelSynthesisPluginManager()
print(HLS_plugin_manager.plugins.names())

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

['clifford.default', 'linear_function.default', 'permutation.acg', 'permutation.basic', 'permutation.default', 'permutation.kms', 'paulievolutiongate.cartan', 'PauliEvolution.cartan2']


#### Test the pass

In [11]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HLSConfig, HighLevelSynthesis
from qiskit.transpiler import PassManager

qc_before = QuantumCircuit(3)
qc_before.append(test_paulievolutiongate, range(3))
print("before: ")
qc_before.draw()

before: 


┌──────────────────────────────────────────────────┐
q_0: ┤0                                                 ├
     │                                                  │
q_1: ┤1 exp(-it (XXI + YYI + ZZI + IXX + IYY + IZZ))(3) ├
     │                                                  │
q_2: ┤2                                                 ├
     └──────────────────────────────────────────────────┘

In [12]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HLSConfig

hls_config = HLSConfig(PauliEvolution=[("cartan2", {})])
pm = PassManager()
pm.append(HighLevelSynthesis(hls_config=hls_config))
qc_after = pm.run(qc_before)
qc_after.draw()

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

Running Cartan Synthesis Plugin...


INFO:qiskit.transpiler.runningpassmanager:Pass: HighLevelSynthesis - 85.09660 (ms)


Optimization terminated successfully.
         Current function value: -2.303234
         Iterations: 15
         Function evaluations: 18
         Gradient evaluations: 18
--- 0.07500696182250977 seconds ---
Optimization Error:
3.878611739499617e-12


┌───────────────────────┐┌─────────────────────────┐»
q_0: ┤0                      ├┤0                        ├»
     │                       ││                         │»
q_1: ┤1 exp(-it YZX)(1.1781) ├┤1 exp(-it ZYX)(-0.49347) ├»
     │                       ││                         │»
q_2: ┤2                      ├┤2                        ├»
     └───────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                        ├┤0                        ├»
«     │                         ││                         │»
«q_1: ┤1 exp(-it XZY)(-0.47282) ├┤1 exp(-it ZXY)(-0.30774) ├»
«     │                         ││                         │»
«q_2: ┤2                        ├┤2                        ├»
«     └─────────────────────────┘└─────────────────────────┘»
«     ┌───────────────────────┐┌─────────────────────────┐ ░ »
«q_0: ┤0                      ├┤0                        ├─░─»
«     │                       ││                         │ ░ »
«q_1: ┤1 exp(-it XYZ)(1.1059) ├┤1 exp(-it YXZ)(-0.86749) ├─░─»
«     │                       ││                         │ ░ »
«q_2: ┤2                      ├┤2                        ├─░─»
«     └───────────────────────┘└─────────────────────────┘ ░ »
«     ┌───────────────────┐┌──────────────────┐┌───────────────────┐ ░ »
«q_0: ┤0                  ├┤0                 ├┤0                  ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_1: ┤1 exp(-it IXX)(-6) ├┤1 exp(-it IYY)(3) ├┤1 exp(-it IZZ)(-3) ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_2: ┤2                  ├┤2                 ├┤2                  ├─░─»
«     └───────────────────┘└──────────────────┘└───────────────────┘ ░ »
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it YXZ)(0.86749) ├┤1 exp(-it XYZ)(-1.1059) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it ZXY)(0.30774) ├┤1 exp(-it XZY)(0.47282) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐
«q_0: ┤0                       ├┤0                       ├
«     │                        ││                        │
«q_1: ┤1 exp(-it ZYX)(0.49347) ├┤1 exp(-it YZX)(-1.1781) ├
«     │                        ││                        │
«q_2: ┤2                       ├┤2                       ├
«     └────────────────────────┘└────────────────────────┘